# 1. YOLO RarePlanes Pre-Processing

This is the first of four tutorial notebooks that implement the YOLOv5 object detection network on the RarePlanes dataset to identify aircraft characteristics of interest. 

In this notebook, you will be creating and detecting a custom class that combines the number of engines and propulsion type. Creating meaningful custom classes can boost model performace in areas of intrest as it forces the model to differentiate in a specific way but, will also find information on how to create your own custom classes or how to use the prebuilt RarePlanes classes with YOLO. 

This ML pipeline uses a modified implementation of the YOLOv5 implementation found [here](https://github.com/ultralytics/yolov5). The RarePlanes dataset can be found [here](https://www.cosmiqworks.org/rareplanes/) and helper functions for the dataset can be found [here](https://github.com/aireveries/RarePlanes). 

In this notebook you will create a custom class to detect, pre-process the dataset images into YOLO format, and create the YOLO specific `.yaml` files. 

In [1]:
import solaris as sol
import numpy as np
import geopandas as gpd
import os
import sys
import pandas as pd
import gdal
import glob
import shapely
import shutil
import datetime
import rasterio 
import argparse
from solaris.vector.mask import footprint_mask
from solaris.vector.polygon import geojson_to_px_gdf, get_overlapping_subset
from solaris.utils.core import _check_gdf_load
from tqdm import tqdm

/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/solaris/lib/pyt

### Create Custom Classes

The following functions come from the RarePlanes GitHub (link above) and are used to create custom classes using any combination of 'role','num_engines', 'propulsion', 'canards', 'num_tail_fins','wing_position', 'wing_type', 'faa_wingspan_class'. More informaiton on the classes and the dataset can be found [here](https://medium.com/the-downlinq/rareplanes-dataset-paper-and-code-release-5b0cba300a0d)

In [4]:
### From https://github.com/aireveries/RarePlanes/blob/master/tools/create_custom_classes.py ###

def count_unique_index(df, by):
    return df.groupby(by).size().reset_index().rename(columns={0: 'count'})

def create_custom_classes(all_annotations_geojson, geojson_dir, output_path, category_attributes):
    """ parse the geojson files and create custom classes based upon
    unique variatons of the RarePlanes attributes.
        -all_annotations_geojson (str): The path to the
        `RarePlanes_Public_All_Annotations.geojson` file.
        - geojson_dir (str): directory containing the geojson files
        for individual images or tiles
        - output_path (str): directory to output the customized geojsons. Need to provide the absolute path.
        - category_attributes (list): A list of attributes to combine
        to create a custom class.  Choose any combintaion of the following:
        ['role','num_engines', 'propulsion', 'canards', 'num_tail_fins',
       'wing_position', 'wing_type', 'faa_wingspan_class']
    :returns
        - new geojsons with a custom_id for each combination of unique
        attributes.
        -A lookup table for each classes custom_id.
    """
    os.makedirs(output_path, exist_ok=True)
    gdf = gpd.read_file(all_annotations_geojson)
    lookup_gdf = count_unique_index(gdf, category_attributes)
    lookup_gdf['custom_id'] = list(range(1, len(lookup_gdf) + 1))
    lookup_gdf.drop(columns=['count'], inplace=True)
    lookup_gdf.to_csv(os.path.join(output_path, "custom_class_lookup.csv"))
    os.chdir(geojson_dir)
    geojsons = glob.glob("*.geojson")
    for geojson in tqdm(geojsons):
        gdf = gpd.read_file(geojson)
        gdf = pd.merge(gdf, lookup_gdf, on=category_attributes, how='left')
        gdf.to_file(os.path.join(output_path, geojson), driver="GeoJSON", encoding='utf-8')

In [18]:
## We recommend you leave these paths static for training/testing as yolo requires files to be in specific locations.
## Feel free to customize classes using any combination of using any combination of 'role','num_engines', 'propulsion', 'canards', 'num_tail_fins','wing_position', 'wing_type', or 'faa_wingspan_class'

#geojsons
all_annotations_geojson = '/home/ubuntu/src/yolo_planes/wdata/RarePlanes_Public_All_Annotations.geojson'
geojson_dir_train = '/home/ubuntu/src/yolo_planes/wdata/train/geojson_aircraft_tiled/'
geojson_dir_test = '/home/ubuntu/src/yolo_planes/wdata/test/geojson_aircraft_tiled/'

#output paths
output_path_train_one = '/home/ubuntu/src/yolo_planes/geojsons_train/yolo_class_one'
output_path_test_one = '/home/ubuntu/src/yolo_planes/geojsons_test/yolo_class_one'

#classes to include in custom class (edit therese with any others you would like to add/subtract)
class_one = ['num_engines', 'propulsion', 'role']

#train
create_custom_classes(all_annotations_geojson, geojson_dir_train, output_path_train_one, class_one)

#test
create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_one, class_one)

100%|██████████| 2710/2710 [01:10<00:00, 38.37it/s]


### Creating YOLO `.yaml` Files

In the data `.yaml`, you edit the paths to the images at the top, both train and val (test) images as well as the number of classes and the class list. Save the new classes to edit the `./data/class_one.yaml` file, we print the class list and number of classes to edit your yaml files below. We've already taken care of this for the number of engines and propulsion type custom class, but if you would like to use a different one it should look something like the example yaml below

```
train: ../class_one/images/train/
val: ../class_one/images/val/

# number of classes
nc: 8

# class names
names: ['0,0,unpowered,1','1,1,jet,2','2,1,propeller,3','3,2,jet,4','4,2,propeller,5','5,3,jet,6','6,4,jet,7','7,4,propeller,8']
```

Additionally, change the number of classes in the `./models/yolov5l.yaml`. In this case, the number after `nc:` at the top of the `yolov5l.yaml` file to the number of classes you print in the next block. 

In [1]:
df = pd.read_csv(os.path.join(output_path_train_one,'custom_class_lookup.csv'))
classes = []
for _, row in df[df.columns[1:-1]].iterrows():
    string = ""
    for item in row:
        string += str(item) + ","
    classes.append(string[:-1])
print(classes)
print("number of classes", len(classes))

NameError: name 'pd' is not defined

Once you've run this block, you can find the custom geojsons in the directory `yolo_class_one` as well as the class lookup table.

### Creating YOLO Labels

The following solaris function is used to convert the tiled georeferenced images to YOLOv5 lables whcich contain space delimted class id and location information of the objects of interst in each image (in this case planes). 

In [2]:
## Training Labels to YOLO
def gdf_to_yolo(geodataframe, image, output_dir, column='single_id', im_size=(512, 512), min_overlap=0):
    """
    Convert a geodataframe containing polygons to yolo/yolt format.
    Arguments
    ---------
    geodataframe : str
        Path to a :class:`geopandas.GeoDataFrame` with a column named
        ``'geometry'``.  Can be created from a geojson with labels for unique
        objects. Can be converted to this format with
        ``geodataframe=gpd.read_file("./xView_30.geojson")``.
    image : str
        Path to a georeferenced image (ie a GeoTIFF or png created with GDAL)
        that geolocates to the same geography as the `geojson`(s). This function will also
        accept a :class:`osgeo.gdal.Dataset` or :class:`rasterio.DatasetReader`
        with georeferencing information in this argument.
    output_dir : str
        Path to an output directory where all of the yolo readable text files
        will be placed.
    column : str, optional
        The column name that contians an unique integer id for each of object
        class.
    im_size : tuple, optional
        A tuple specifying the x and y heighth of a an image.  If specified as
        ``(0,0)`` (the default,) then the size is determined automatically.
    min_overlap : float, optional
        A float value ranging from 0 to 1.  This is a percantage.  If a polygon
        does not overlap the image by at least min_overlap, the polygon is
        discarded.  i.e. 0.66 = 66%. Default value of 0.66.
    Returns
    -------
    gdf : :class:`geopandas.GeoDataFrame`.
        The txt file will be written to the output_dir, however the the output
        gdf itself is returned.
    """
    if im_size == (0, 0):
        imsize_extract = rasterio.open(image).read()
        if len(imsize_extract.shape) == 3:
            im_size = (imsize_extract.shape[1], imsize_extract.shape[2])
        else:
            im_size = (imsize_extract.shape[0], imsize_extract.shape[1])
    [x0, y0, x1, y1] = [0, 0, im_size[0], im_size[1]]
    out_coords = [[x0, y0], [x0, y1], [x1, y1], [x1, y0]]
    points = [shapely.geometry.Point(coord) for coord in out_coords]
    pix_poly = shapely.geometry.Polygon([[p.x, p.y] for p in points])
    dw = 1. / im_size[0]
    dh = 1. / im_size[1]
    header = ["column_sub_1", "x", "y", "w", "h"]
    if os.path.isdir(output_dir) is False:
        os.mkdir(output_dir)    
    img_name = image.split('/')[-1]
    output = os.path.join(output_dir, img_name.split('.png')[0] + ".txt")
    gdf = geojson_to_px_gdf(geodataframe, image, precision=None)
    gdf['column_sub_1'] = gdf[column] - 1
    gdf['area'] = gdf['geometry'].area
    gdf['intersection'] = (
        gdf['geometry'].intersection(pix_poly).area / gdf['area'])
    gdf = gdf[gdf['area'] != 0]
    gdf = gdf[gdf['intersection'] >= min_overlap]
    if not gdf.empty:
        boxy = gdf['geometry'].bounds
        for _,row in boxy.iterrows():
            if row['maxx'] > im_size[0]:
                row['maxx'] = im_size[0]
            if row['minx'] < 0:
                row['minx'] = 0
            if row['maxy'] > im_size[1]:
                row['maxy'] = im_size[1]
            if row['miny'] < 0:
                row['miny'] = 0
        boxy['xmid'] = (boxy['minx'] + boxy['maxx']) / 2.0
        boxy['ymid'] = (boxy['miny'] + boxy['maxy']) / 2.0
        boxy['w0'] = (boxy['maxx'] - boxy['minx'])
        boxy['h0'] = (boxy['maxy'] - boxy['miny'])
        boxy['x'] = boxy['xmid'] * dw
        boxy['y'] = boxy['ymid'] * dh
        boxy['w'] = boxy['w0'] * dw
        boxy['h'] = boxy['h0'] * dh
        if not boxy.empty:
            gdf = gdf.join(boxy)
        gdf.to_csv(path_or_buf=output, sep=' ', columns=header, index=False, header=False)
    return gdf

In [3]:
truth_dir_train_name = '/home/ubuntu/src/yolo_planes/class_one/images/train'
truth_dir_test_name = '/home/ubuntu/src/yolo_planes/class_one/images/val'
truth_dir_train = os.fsencode(truth_dir_train_name)
truth_dir_test = os.fsencode(truth_dir_test_name)

geo_dir_train_one_name = '/home/ubuntu/src/yolo_planes/geojsons_train/yolo_class_one'
geo_dir_train_one = os.fsencode(geo_dir_train_one_name)

geo_dir_test_one_name = '/home/ubuntu/src/yolo_planes/geojsons_test/yolo_class_one'
geo_dir_test_one = os.fsencode(geo_dir_test_one_name)

output_dir_train_one = '/home/ubuntu/src/yolo_planes/class_one/labels/train'
os.makedirs(output_dir_train_one, exist_ok = True)

output_dir_test_one = '/home/ubuntu/src/yolo_planes/class_one/labels/val'
os.makedirs(output_dir_test_one, exist_ok = True)

OSError: [Errno 45] Operation not supported: '/home/ubuntu'

In [4]:
geo_dir_name = geo_dir_train_one_name
geo_dir = geo_dir_train_one
truth_dir_name = truth_dir_train_name
truth_dir = truth_dir_train
output_dir = output_dir_train_one

for file in tqdm(os.listdir(geo_dir)):
    filename = os.fsdecode(file)
    if filename.endswith(".geojson"):
        geodataframe = gpd.read_file(geo_dir_name + '/' + filename)
        image_name = filename.replace('.geojson', '')     
        
        for truth in os.listdir(truth_dir):
            truth_name = os.fsdecode(truth)
            if truth_name.endswith(".png"):
                check_name = truth_name.replace('.png', '')
                if (check_name == image_name):
                    image = truth_dir_name + '/' + truth_name 
                    gdf_to_yolo(geodataframe, image, output_dir=output_dir, column='custom_id', im_size=(512, 512), min_overlap=0)

NameError: name 'tqdm' is not defined

In [5]:
geo_dir_name = geo_dir_test_one_name
geo_dir = geo_dir_test_one
truth_dir_name = truth_dir_test_name
truth_dir = truth_dir_test
output_dir = output_dir_test_one

for file in tqdm(os.listdir(geo_dir)):
    filename = os.fsdecode(file)
    if filename.endswith(".geojson"):
        geodataframe = gpd.read_file(geo_dir_name + '/' + filename)
        image_name = filename.replace('.geojson', '')     
        
        for truth in os.listdir(truth_dir):
            truth_name = os.fsdecode(truth)
            if truth_name.endswith(".png"):
                check_name = truth_name.replace('.png', '')
                if (check_name == image_name):
                    image = truth_dir_name + '/' + truth_name 
                    gdf_to_yolo(geodataframe, image, output_dir=output_dir, column='custom_id', im_size=(512, 512), min_overlap=0)

NameError: name 'output_dir_test_one' is not defined

Once you've run these three blocks, you can find the labels in the class_one labels directories—val refers to testing data.

Now, please head to the notebook titled `2_yolo_train.ipynb`.